### For each zipcode find which other zipcode they are most and least similar to.  Does this change over time?  If so, how.

We can think of zip code similarities on the basis of complaint type. If two zipdcodes have almost same complaint type count then probabliity of the zip code to be similar is high.
First identify top complaints in total 

In [6]:
# Load it into a Dataframe using pandas
import pandas as pd
path = 'nyc311_filter.csv'
df = pd.read_csv(path, low_memory=False, na_values = ['NO CLUE', 'N/A', '0', 'NaN'])
df.head()

unique_key            created_date             closed_date agency  \
0    20122810  03/28/2011 11:20:53 AM  04/13/2011 10:40:27 AM    DOF   
1    20122811  03/28/2011 11:12:27 PM  03/29/2011 05:07:13 AM   NYPD   
2    20122812  03/28/2011 04:12:08 PM  03/28/2011 08:03:39 PM    DOF   
3    20122813  03/28/2011 06:25:54 PM  03/28/2011 07:25:59 PM    DOF   
4    20122814  03/28/2011 04:16:51 PM  03/28/2011 07:52:23 PM    DOF   

                                   agency_name   complaint_type  \
0  Senior Citizen Rent Increase Exemption Unit            SCRIE   
1              New York City Police Department  Illegal Parking   
2  Senior Citizen Rent Increase Exemption Unit            SCRIE   
3  Senior Citizen Rent Increase Exemption Unit            SCRIE   
4  Senior Citizen Rent Increase Exemption Unit            SCRIE   

                      descriptor    location_type  incident_zip  \
0               SCRIE Assistance  Address Unknown           NaN   
1  Posted Parking Sign Violation  Street/Sidewalk       11216.0   
2               SCRIE Assistance   Senior Address       10012.0   
3               SCRIE Assistance   Senior Address       10456.0   
4               SCRIE Assistance   Senior Address       10023.0   

       incident_address  ... vehicle_type taxi_company_borough  \
0                   NaN  ...          NaN                  NaN   
1  644 LAFAYETTE AVENUE  ...          NaN                  NaN   
2                   NaN  ...          NaN                  NaN   
3                   NaN  ...          NaN                  NaN   
4                   NaN  ...          NaN                  NaN   

  taxi_pick_up_location bridge_highway_name bridge_highway_direction  \
0                   NaN                 NaN                      NaN   
1                   NaN                 NaN                      NaN   
2                   NaN                 NaN                      NaN   
3                   NaN                 NaN                      NaN   
4                   NaN                 NaN                      NaN   

  road_ramp bridge_highway_segment   latitude  longitude  \
0       NaN                    NaN        NaN        NaN   
1       NaN                    NaN  40.690123 -73.949135   
2       NaN                    NaN        NaN        NaN   
3       NaN                    NaN        NaN        NaN   
4       NaN                    NaN        NaN        NaN   

                                  location  
0                                      NaN  
1  (40.69012317171803, -73.94913492072885)  
2                                      NaN  
3                                      NaN  
4                                      NaN  

[5 rows x 41 columns]

In [7]:
df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [8]:
df.drop(df.columns[[9,10,11,12,13,14,15,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37]], axis=1, inplace=True)

In [9]:
# Drop null values from the whole dataset
df = df.dropna()

In [10]:
df.head()

unique_key            created_date             closed_date agency  \
1     20122811  03/28/2011 11:12:27 PM  03/29/2011 05:07:13 AM   NYPD   
11    20122821  03/28/2011 10:57:31 PM  05/19/2011 01:13:06 PM  DOHMH   
12    20122822  03/28/2011 12:13:02 PM  05/19/2011 01:12:55 PM  DOHMH   
13    20122823  03/28/2011 04:23:23 PM  05/19/2011 01:13:02 PM  DOHMH   
14    20122824  03/28/2011 11:14:53 AM  04/04/2011 12:46:58 PM  DOHMH   

                                agency_name      complaint_type  \
1           New York City Police Department     Illegal Parking   
11  Department of Health and Mental Hygiene  Indoor Air Quality   
12  Department of Health and Mental Hygiene  Indoor Air Quality   
13  Department of Health and Mental Hygiene  Indoor Air Quality   
14  Department of Health and Mental Hygiene  Indoor Air Quality   

                       descriptor                 location_type  incident_zip  \
1   Posted Parking Sign Violation               Street/Sidewalk       11216.0   
11         Dust from Construction  3+ Family Apartment Building       10009.0   
12         Dust from Construction  3+ Family Apartment Building       10009.0   
13         Dust from Construction  3+ Family Apartment Building       11360.0   
14         Dust from Construction  3+ Family Apartment Building       11205.0   

        city           bbl   latitude  longitude  \
1   BROOKLYN  3.017890e+09  40.690123 -73.949135   
11  NEW YORK  1.003910e+09  40.726053 -73.979818   
12  NEW YORK  1.003910e+09  40.726053 -73.979818   
13   BAYSIDE  4.058938e+09  40.779012 -73.782085   
14  BROOKLYN  3.019170e+09  40.690711 -73.966360   

                                    location  
1    (40.69012317171803, -73.94913492072885)  
11  (40.726053362722304, -73.97981762009752)  
12  (40.726053362722304, -73.97981762009752)  
13  (40.779012083546895, -73.78208491752011)  
14   (40.69071138727672, -73.96636009530222)

In [11]:
# How this change over time
# Create a function that gets the year of a datetime string
def getYear(s):
    first_split = s.split(' ')[0]
    return int(first_split.split('/')[-1])

In [13]:
# Now create a new column named year and store the values of the above function
df['created_year']= df['created_date'].apply(lambda x: getYear(x))
df['closed_year']= df['closed_date'].apply(lambda x: getYear(x))

In [14]:
# Keep only 2010 to 2018 data
df = df[df['closed_year'].isin(range(2010,2019))]

In [15]:
# Create a new csv
df.to_csv('311_cleaned_2.csv', index=False)

In [16]:
df.head()

unique_key            created_date             closed_date agency  \
1     20122811  03/28/2011 11:12:27 PM  03/29/2011 05:07:13 AM   NYPD   
11    20122821  03/28/2011 10:57:31 PM  05/19/2011 01:13:06 PM  DOHMH   
12    20122822  03/28/2011 12:13:02 PM  05/19/2011 01:12:55 PM  DOHMH   
13    20122823  03/28/2011 04:23:23 PM  05/19/2011 01:13:02 PM  DOHMH   
14    20122824  03/28/2011 11:14:53 AM  04/04/2011 12:46:58 PM  DOHMH   

                                agency_name      complaint_type  \
1           New York City Police Department     Illegal Parking   
11  Department of Health and Mental Hygiene  Indoor Air Quality   
12  Department of Health and Mental Hygiene  Indoor Air Quality   
13  Department of Health and Mental Hygiene  Indoor Air Quality   
14  Department of Health and Mental Hygiene  Indoor Air Quality   

                       descriptor                 location_type  incident_zip  \
1   Posted Parking Sign Violation               Street/Sidewalk       11216.0   
11         Dust from Construction  3+ Family Apartment Building       10009.0   
12         Dust from Construction  3+ Family Apartment Building       10009.0   
13         Dust from Construction  3+ Family Apartment Building       11360.0   
14         Dust from Construction  3+ Family Apartment Building       11205.0   

        city           bbl   latitude  longitude  \
1   BROOKLYN  3.017890e+09  40.690123 -73.949135   
11  NEW YORK  1.003910e+09  40.726053 -73.979818   
12  NEW YORK  1.003910e+09  40.726053 -73.979818   
13   BAYSIDE  4.058938e+09  40.779012 -73.782085   
14  BROOKLYN  3.019170e+09  40.690711 -73.966360   

                                    location  created_year  closed_year  
1    (40.69012317171803, -73.94913492072885)          2011         2011  
11  (40.726053362722304, -73.97981762009752)          2011         2011  
12  (40.726053362722304, -73.97981762009752)          2011         2011  
13  (40.779012083546895, -73.78208491752011)          2011         2011  
14   (40.69071138727672, -73.96636009530222)          2011         2011

In [17]:
# find top complaint types
top_complaint = df['complaint_type'].groupby(df['complaint_type']).count(
).sort_values(ascending=False).reset_index(name='count')
top_complaint.head()

complaint_type    count
0  Noise  Residential  1073747
1       HEATHOT WATER   983313
2             HEATING   573033
3    Blocked Driveway   551327
4            PLUMBING   492611

In [18]:
# Keep noise related data in a seperate dataframe
df_noise = df[df['complaint_type'].str.contains("Noise", na=False)]
df_noise['complaint_type'].unique()

array(['Noise  Residential', 'Noise  StreetSidewalk', 'Noise  Vehicle',
       'Noise  Commercial', 'Noise  Helicopter',
       'Noise  House of Worship', 'Noise  Park', 'Noise Survey',
       'Collection Truck Noise'], dtype=object)

In [19]:
# list the names of zips
# Create a list of unique values of districts excluding nulls
zips = [i for i in df_noise['incident_zip'].unique() if pd.notnull(i)]
print('The %d zips of NYC are: \n%s' % (len(zips), zips))

The 213 zips of NYC are: 
[10310.0, 11226.0, 11377.0, 10465.0, 11373.0, 10452.0, 11204.0, 11370.0, 10462.0, 10473.0, 11206.0, 10040.0, 11221.0, 11374.0, 10467.0, 10009.0, 10026.0, 11212.0, 10039.0, 11693.0, 11209.0, 10468.0, 10472.0, 10029.0, 11213.0, 10035.0, 10027.0, 11432.0, 10014.0, 11205.0, 10023.0, 11217.0, 10013.0, 10019.0, 10002.0, 10028.0, 11372.0, 11435.0, 11413.0, 11207.0, 11229.0, 10022.0, 11214.0, 10469.0, 11354.0, 11415.0, 10454.0, 10458.0, 10030.0, 10461.0, 11106.0, 10301.0, 11419.0, 10025.0, 10453.0, 10034.0, 11237.0, 10457.0, 10455.0, 11385.0, 10304.0, 11208.0, 10459.0, 11230.0, 10032.0, 11216.0, 11355.0, 11219.0, 11429.0, 10001.0, 11691.0, 10451.0, 11234.0, 10031.0, 10463.0, 11236.0, 11358.0, 11422.0, 10037.0, 11104.0, 10003.0, 10016.0, 11203.0, 11210.0, 11368.0, 10128.0, 11220.0, 11201.0, 10011.0, 11224.0, 10466.0, 10456.0, 11365.0, 11225.0, 11211.0, 10474.0, 11215.0, 11222.0, 11218.0, 10471.0, 11103.0, 11238.0, 11412.0, 10303.0, 10460.0, 11004.0, 11233.0, 11434.0, 1

In [20]:
noise_freq = df_noise['complaint_type'].groupby(df_noise['complaint_type']).count(
).sort_values(ascending=False).reset_index(name='count')
noise_freq

complaint_type    count
0       Noise  Residential  1073747
1        Noise  Commercial   199869
2    Noise  StreetSidewalk   194617
3           Noise  Vehicle    79504
4             Noise Survey    13417
5              Noise  Park     5064
6        Noise  Helicopter     4533
7  Noise  House of Worship     3962
8   Collection Truck Noise      713

In [21]:
# Save the five most significant noise incidents in a list
noise_list = list(noise_freq['complaint_type'].values)[-5:]
noise_list

['Noise Survey',
 'Noise  Park',
 'Noise  Helicopter',
 'Noise  House of Worship',
 'Collection Truck Noise']

In [23]:
#We will explore which zips have the most noise complaints.
# Keep only the Category and the zips column
df_noise_zips = df_noise.filter(items=['complaint_type', 'incident_zip'])
df_noise_zips = df_noise_zips[df_noise_zips['incident_zip'].isin(zips)]

# List the number of complaints per zips
complaint_freq_dist = df_noise_zips.groupby('incident_zip')['incident_zip'].count(
).reset_index(name='count').sort_values(by='count', ascending=0)

# Print the zips with the most noise complaints
print('The zips with the most noise complains are:\n')
complaint_freq_dist = complaint_freq_dist.reset_index().drop('index',axis=1)
complaint_freq_dist.head(10)

The zips with the most noise complains are:



incident_zip  count
0       11226.0  36045
1       10031.0  33002
2       10467.0  31054
3       10032.0  29746
4       10040.0  29684
5       10468.0  29214
6       10034.0  26567
7       10452.0  25670
8       10456.0  25610
9       10458.0  25315

In [24]:
# Keep the most noisy districts in a seperate list.
noisy_zips = [i for i in complaint_freq_dist['incident_zip'][:10]]
print("Most noisy zips", noisy_zips)

Most noisy zips [11226.0, 10031.0, 10467.0, 10032.0, 10040.0, 10468.0, 10034.0, 10452.0, 10456.0, 10458.0]


In [25]:
#focus only on noisy zips
df_noise_focus = df_noise[df_noise['incident_zip'].isin(noisy_zips)]

In [26]:
df_noise_focus.head()

unique_key            created_date             closed_date agency  \
423     20123360  03/28/2011 08:53:56 AM  03/28/2011 10:40:26 AM   NYPD   
427     20123364  03/28/2011 11:53:22 PM  03/29/2011 12:55:03 AM   NYPD   
433     20123369  03/29/2011 12:09:29 AM  03/29/2011 01:26:40 AM   NYPD   
438     20123374  03/28/2011 11:12:10 PM  03/29/2011 12:31:39 AM   NYPD   
2325    30932618  06/25/2015 12:55:27 PM  06/25/2015 02:28:23 PM   NYPD   

                          agency_name         complaint_type  \
423   New York City Police Department     Noise  Residential   
427   New York City Police Department     Noise  Residential   
433   New York City Police Department     Noise  Residential   
438   New York City Police Department     Noise  Residential   
2325  New York City Police Department  Noise  StreetSidewalk   

            descriptor               location_type  incident_zip      city  \
423   Loud Music/Party  Residential Building/House       11226.0  BROOKLYN   
427   Loud Music/Party  Residential Building/House       10452.0     BRONX   
433   Loud Music/Party  Residential Building/House       10040.0  NEW YORK   
438    Loud Television  Residential Building/House       10467.0     BRONX   
2325  Loud Music/Party             Street/Sidewalk       10468.0     BRONX   

               bbl   latitude  longitude  \
423   3.050820e+09  40.651624 -73.960235   
427   2.028410e+09  40.838477 -73.916792   
433   1.021740e+09  40.861169 -73.927678   
438   2.043370e+09  40.858516 -73.869540   
2325  2.033100e+09  40.872514 -73.887534   

                                     location  created_year  closed_year  
423   (40.65162381847338, -73.96023526159061)          2011         2011  
427   (40.83847687349654, -73.91679220209002)          2011         2011  
433    (40.8611692009135, -73.92767807904268)          2011         2011  
438   (40.85851613343683, -73.86954011309079)          2011         2011  
2325  (40.87251382811215, -73.88753385727848)          2015         2015

In [27]:
df_noise_focus = df_noise_focus.dropna(subset=['latitude'])
df_noise_focus = df_noise_focus.dropna(subset=['longitude'])

In [29]:
df_noise_focus.plot(kind='hexbin', x='longitude', y='latitude', gridsize=40,
    colormap = 'jet',mincnt=1,title = 'Noise Complaints concentration zips', figsize=(10,6)).axis('equal')

(-73.97173593629704, -73.85721528506248, 40.63438479278872, 40.886431438151796)

In [32]:
mergedStuff = pd.merge(complaint_freq_dist, df_new, on=['incident_zip'], how='inner')
mergedStuff.head()

incident_zip  count  unique_key            created_date  \
0       11226.0  36045    20123360  03/28/2011 08:53:56 AM   
1       11226.0  36045    30934881  06/25/2015 05:05:36 PM   
2       11226.0  36045    20127486  03/30/2011 12:51:35 AM   
3       11226.0  36045    20127496  03/30/2011 01:15:02 AM   
4       11226.0  36045    20127503  03/29/2011 10:12:11 PM   

              closed_date agency                      agency_name  \
0  03/28/2011 10:40:26 AM   NYPD  New York City Police Department   
1  06/25/2015 11:13:56 PM   NYPD  New York City Police Department   
2  03/30/2011 03:34:38 AM   NYPD  New York City Police Department   
3  03/30/2011 02:06:37 AM   NYPD  New York City Police Department   
4  03/29/2011 11:19:08 PM   NYPD  New York City Police Department   

          complaint_type        descriptor               location_type  \
0     Noise  Residential  Loud Music/Party  Residential Building/House   
1  Noise  StreetSidewalk  Loud Music/Party             Street/Sidewalk   
2     Noise  Residential  Banging/Pounding  Residential Building/House   
3     Noise  Residential  Banging/Pounding  Residential Building/House   
4     Noise  Residential  Banging/Pounding  Residential Building/House   

       city           bbl   latitude  longitude  \
0  BROOKLYN  3.050820e+09  40.651624 -73.960235   
1  BROOKLYN  3.051280e+09  40.646708 -73.953806   
2  BROOKLYN  3.052350e+09  40.634901 -73.963254   
3  BROOKLYN  3.049300e+09  40.645142 -73.948665   
4  BROOKLYN  3.051250e+09  40.646331 -73.958660   

                                   location  created_year  closed_year  
0   (40.65162381847338, -73.96023526159061)          2011         2011  
1    (40.64670825331842, -73.9538058153507)          2015         2015  
2   (40.63490084624346, -73.96325367646664)          2011         2011  
3   (40.645141546136394, -73.9486647235038)          2011         2011  
4  (40.646331320458025, -73.95866006025375)          2011         2011

In [42]:
data = {'incident_zip': mergedStuff['incident_zip'], 
        'count': mergedStuff['count'], 
        'created_year': mergedStuff['created_year']} 
  
by_zip = pd.DataFrame(data) 
by_zip = by_zip.drop_duplicates()

In [44]:
by_zip.head(20)

incident_zip  count  created_year
0           11226.0  36045          2011
1           11226.0  36045          2015
9           11226.0  36045          2017
2323        11226.0  36045          2012
2424        11226.0  36045          2016
5421        11226.0  36045          2013
6865        11226.0  36045          2014
11668       11226.0  36045          2010
31132       10031.0  33002          2011
31142       10031.0  33002          2015
31194       10031.0  33002          2017
32956       10031.0  33002          2016
33140       10031.0  33002          2012
35911       10031.0  33002          2013
37509       10031.0  33002          2014
40218       10031.0  33002          2010
58645       10467.0  31054          2011
58651       10467.0  31054          2015
58743       10467.0  31054          2017
61099       10467.0  31054          2012

#### We can see from the above data that similarity between zipcodes doesn't change over year